In [293]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [294]:
# path to playlist file
root_path = !pwd
root_path = str(root_path[0])

In [295]:
# filename 
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [296]:
# path + filename
fpath_name = f"{filepath}{filename}"
# print(fpath_name)

In [297]:
# Open file of playlist and obtain fields of json file 
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

In [298]:
# # Explore fields in json file
# print("File keys: ", list(data.keys()))
# print("info: ",data[list(data.keys())[0]])
# #print(f"{list(data.keys())[1]} : ",data[list(data.keys())[1]])
# print("playlists: ",list(data['playlists'][0].keys()))
# print("tracks: ", list(data['playlists'][0]['tracks'][0].keys()))

In [299]:
# Extract track features
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

In [300]:
# Construct keys of playlist
keys = list(keys)
keys.remove('tracks')
print(" Playlist keys: ", keys)

 Playlist keys:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [301]:
# Create dataframe with track and playlist info
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'], record_prefix = 'track_')
music_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


In [302]:
# Check column values 
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [303]:
subset_df = music_df

In [304]:
# Select indexes of playlists that have a description
# playlists_id_with_description = music_df[~music_df['playlist_description'].isna()]['playlist_pid'].unique()
# print(playlists_id_with_description)

In [305]:
# Working only with playlists that contain a "description"
# subset_df = music_df[music_df['playlist_pid'].isin(playlists_id_with_description)]
# print(subset_df['playlist_pid'].unique())

In [306]:
num_records = len(subset_df)
num_unique_records = len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique())
print('Number of table records: ', num_records)
print('Number of unique artist, song pairs: ', num_unique_records )
print('Number of repeated songs: ', num_records - num_unique_records )
print('Number of playlists: ', len(subset_df['playlist_pid'].unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

Number of table records:  67503
Number of unique artist, song pairs:  34250
Number of repeated songs:  33253
Number of playlists:  1000
Number of unique artists:  9722


In [375]:
# Function to select track_uri ONLY Nrecords records for now
Nrecords = 1000
def get_features(df, Nrecords):
    indexes = []
    audio_features = []
    track_popularity = []
    artist_genre = []
    artist_popularity = []
    album_popularity = []
    start = 0
    for row_index, row in df.iloc[0:Nrecords].iterrows():
        #print(row_index, row['track_track_uri'])
        track_uri = row['track_track_uri']
        artist_uri = row['track_artist_uri']
        album_uri = row['track_album_uri']
        if start == 0:
            keys = spotify.audio_features(tracks=track_uri)[0].keys()
        start += 1
    
        track_popularity.append(spotify.track(track_uri)['popularity'])
        
        indexes.append(row_index) 
        audio_features.append(spotify.audio_features(tracks=track_uri)[0].values())
        
        artist_results = spotify.artist(artist_uri)
        
        artist_genre.append(artist_results['genres'])
        artist_popularity.append(artist_results['popularity'])
        #print(artist_results['genres'], artist_results['popularity'])
        album_results = spotify.album(album_uri)
        album_popularity.append(album_results['popularity'])
        
        
    features_df = pd.DataFrame(audio_features, columns=keys)
    features_df['song_popularity'] = track_popularity
    features_df['artist_genre'] = artist_genre
    features_df['artist_popularity']  = artist_popularity
    features_df['album_popularity'] = album_popularity
    features_df['index'] = indexes
    features_df.set_index('index', inplace=True)
    return features_df

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [376]:
audio_sp_df = get_features(subset_df)
audio_sp_df.index.name = None
audio_sp_df.drop(['type', 'id', 'track_href', 'analysis_url'], inplace=True, axis=1)
audio_sp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,4,84,"[dance pop, pop]",80,78
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,4,21,"[pop, r&b]",87,17
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267,4,79,"[dance pop, pop]",79,77
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.799,0.586,11,-6.735,1,0.0377,0.162000,0.000002,0.1930,0.549,120.970,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,227833,4,51,"[pop, uk dance, uk funky]",74,67
996,0.543,0.668,0,-7.396,1,0.0272,0.000724,0.000080,0.3220,0.428,93.009,spotify:track:5eqz8gTDU13s7x2DZKiQA9,227267,4,0,"[ccm, christian alternative rock, christian mu...",49,0
997,0.789,0.555,0,-5.900,1,0.0410,0.026000,0.000135,0.2150,0.713,93.229,spotify:track:3V3iy4K6paycRmTyrjQVwi,223405,4,46,[neo mellow],49,44
998,0.705,0.780,7,-6.267,1,0.0805,0.456000,0.001520,0.1150,0.457,142.028,spotify:track:0qt5f5EL92o8Snzopsv0en,218228,4,72,"[australian indie, folk-pop]",61,65


In [378]:
# Parameters to use for the model
audio_sp_df.keys()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity'],
      dtype='object')

In [379]:
check_df = subset_df.iloc[0:Nrecords].merge(audio_sp_df, how='left' , left_on = 'track_track_uri', right_on='uri')
check_df.drop(['track_pos', 'uri', 'mode', 'playlist_duration_ms','playlist_num_albums','playlist_num_artists',  'track_artist_uri', 'track_album_uri', 'track_duration_ms','playlist_num_followers', 'playlist_num_edits', 'playlist_collaborative', 'playlist_modified_at', 'playlist_num_tracks'], inplace = True, axis=1)
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.000002,0.1930,0.549,120.970,227833,4,51,"[pop, uk dance, uk funky]",74,67
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.000080,0.3220,0.428,93.009,227267,4,0,"[ccm, christian alternative rock, christian mu...",49,0
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.000135,0.2150,0.713,93.229,223405,4,46,[neo mellow],49,44
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.001520,0.1150,0.457,142.028,218228,4,72,"[australian indie, folk-pop]",61,65


In [380]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000
mean,0.589546,0.674247,5.080827,-7.176674,0.074820,0.192634,0.109175,0.186292,0.504421,123.339521,239965.338346,3.961466,31.891917,61.795113,33.063910
std,0.159334,0.193272,3.638284,3.113554,0.078974,0.252967,0.250037,0.144331,0.221861,28.841298,64300.169311,0.283474,31.071195,17.728944,31.521106
min,0.084900,0.013400,0.000000,-29.917000,0.023200,0.000010,0.000000,0.023200,0.031100,43.509000,65306.000000,1.000000,0.000000,0.000000,0.000000
25%,0.482750,0.544750,2.000000,-8.799000,0.033900,0.011175,0.000000,0.093300,0.343750,100.312250,201437.750000,4.000000,0.000000,53.000000,0.000000
50%,0.597500,0.707000,5.000000,-6.662000,0.045050,0.068950,0.000079,0.127500,0.506000,120.970000,230068.500000,4.000000,30.500000,64.000000,34.000000
75%,0.705250,0.825000,8.000000,-4.979000,0.075350,0.294000,0.017400,0.242500,0.676000,142.099500,268443.500000,4.000000,63.000000,74.000000,64.000000
max,0.979000,0.986000,11.000000,-1.029000,0.563000,0.989000,0.991000,0.965000,0.975000,210.857000,658987.000000,5.000000,86.000000,95.000000,89.000000


In [381]:
# Make album popularity equal to artist popularity if album popularity is 0
check_df['album_popularity'] = np.where(check_df['album_popularity'] == 0, check_df['artist_popularity'], check_df['album_popularity'])
check_df['song_popularity'] = np.where(check_df['song_popularity'] == 0, check_df['artist_popularity'], check_df['song_popularity'])
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.000002,0.1930,0.549,120.970,227833,4,51,"[pop, uk dance, uk funky]",74,67
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.000080,0.3220,0.428,93.009,227267,4,49,"[ccm, christian alternative rock, christian mu...",49,49
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.000135,0.2150,0.713,93.229,223405,4,46,[neo mellow],49,44
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.001520,0.1150,0.457,142.028,218228,4,72,"[australian indie, folk-pop]",61,65


In [382]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000,1064.000000
mean,0.589546,0.674247,5.080827,-7.176674,0.074820,0.192634,0.109175,0.186292,0.504421,123.339521,239965.338346,3.961466,55.217105,61.795113,55.942669
std,0.159334,0.193272,3.638284,3.113554,0.078974,0.252967,0.250037,0.144331,0.221861,28.841298,64300.169311,0.283474,21.588272,17.728944,21.337928
min,0.084900,0.013400,0.000000,-29.917000,0.023200,0.000010,0.000000,0.023200,0.031100,43.509000,65306.000000,1.000000,0.000000,0.000000,0.000000
25%,0.482750,0.544750,2.000000,-8.799000,0.033900,0.011175,0.000000,0.093300,0.343750,100.312250,201437.750000,4.000000,43.000000,53.000000,46.000000
50%,0.597500,0.707000,5.000000,-6.662000,0.045050,0.068950,0.000079,0.127500,0.506000,120.970000,230068.500000,4.000000,60.500000,64.000000,61.000000
75%,0.705250,0.825000,8.000000,-4.979000,0.075350,0.294000,0.017400,0.242500,0.676000,142.099500,268443.500000,4.000000,71.000000,74.000000,71.000000
max,0.979000,0.986000,11.000000,-1.029000,0.563000,0.989000,0.991000,0.965000,0.975000,210.857000,658987.000000,5.000000,95.000000,95.000000,95.000000


In [383]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1063
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_artist_name     1064 non-null   object 
 1   track_track_uri       1064 non-null   object 
 2   track_track_name      1064 non-null   object 
 3   track_album_name      1064 non-null   object 
 4   playlist_name         1064 non-null   object 
 5   playlist_pid          1064 non-null   object 
 6   playlist_description  0 non-null      object 
 7   danceability          1064 non-null   float64
 8   energy                1064 non-null   float64
 9   key                   1064 non-null   int64  
 10  loudness              1064 non-null   float64
 11  speechiness           1064 non-null   float64
 12  acousticness          1064 non-null   float64
 13  instrumentalness      1064 non-null   float64
 14  liveness              1064 non-null   float64
 15  valence              

In [384]:
# NLP workflow
# Need to vectorize string fields
# Use all string fields and combine them in a column containing all the words f
# from collections import Counter

In [385]:
check_df['artist_genre']

0       [dance pop, hip hop, hip pop, neo soul, pop ra...
1                                        [dance pop, pop]
2                                              [pop, r&b]
3                                        [dance pop, pop]
4                     [dance pop, pop rap, reggae fusion]
                              ...                        
1059                            [pop, uk dance, uk funky]
1060    [ccm, christian alternative rock, christian mu...
1061                                         [neo mellow]
1062                         [australian indie, folk-pop]
1063              [emo, modern rock, pop, pop punk, rock]
Name: artist_genre, Length: 1064, dtype: object

In [386]:
#check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', '_') for i in x]))
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', '_').replace('_', '') for i in x]))

In [387]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.000002,0.1930,0.549,120.970,227833,4,51,pop uk dance uk funky,74,67
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.000080,0.3220,0.428,93.009,227267,4,49,ccm christian alternative rock christian music...,49,49
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.000135,0.2150,0.713,93.229,223405,4,46,neo mellow,49,44
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.001520,0.1150,0.457,142.028,218228,4,72,australian indie folkpop,61,65


In [388]:
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join(list(set(x.split()))))

In [389]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,hop rap urban contemporary virginia hip pop ne...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,pop dance,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,pop dance,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,reggae rap pop dance fusion,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.000002,0.1930,0.549,120.970,227833,4,51,pop uk dance funky,74,67
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.000080,0.3220,0.428,93.009,227267,4,49,worship rock music christian alternative ccm,49,49
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.000135,0.2150,0.713,93.229,223405,4,46,neo mellow,49,44
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.001520,0.1150,0.457,142.028,218228,4,72,australian folkpop indie,61,65


In [390]:
# recheck = check.apply(lambda x: list(set(x.split())))
# print(check)
# print(recheck)
# # check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ''.join([i.replace('', '_') for i in x]))

In [391]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,hop rap urban contemporary virginia hip pop ne...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,pop dance,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,pop dance,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,reggae rap pop dance fusion,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.000002,0.1930,0.549,120.970,227833,4,51,pop uk dance funky,74,67
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.000080,0.3220,0.428,93.009,227267,4,49,worship rock music christian alternative ccm,49,49
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.000135,0.2150,0.713,93.229,223405,4,46,neo mellow,49,44
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.001520,0.1150,0.457,142.028,218228,4,72,australian folkpop indie,61,65


In [392]:
label_columns = ['track_track_name', 'artist_genre']
check_df[label_columns]

,track_track_name,artist_genre
0,Lose Control (feat. Ciara & Fat Man Scoop),hop rap urban contemporary virginia hip pop ne...
1,Toxic,pop dance
2,Crazy In Love,pop r&b
3,Rock Your Body,pop dance
4,It Wasn't Me,reggae rap pop dance fusion
...,...,...
1059,Rather Be (feat. Jess Glynne),pop uk dance funky
1060,Remind Me Who I Am,worship rock music christian alternative ccm
1061,Secrets,neo mellow
1062,Geronimo,australian folkpop indie


In [393]:
# Combine 5 fields into one
# string_field = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
string_field = check_df.artist_genre
string_field = string_field.replace({"r\&b": "rhythm blues"}, regex = True)
string_field = string_field.replace({"[^A-Za-z ]+": ""}, regex = True)
#print('last song: ',string_field.tail())
# test_song = string_field[0:int(len(string_field)*0.4)]
# print(test_song)
#fraction = 0.0
#string_field = string_field[int(len(string_field)*fraction):]
string_field

0       hop rap urban contemporary virginia hip pop ne...
1                                               pop dance
2                                        pop rhythm blues
3                                               pop dance
4                             reggae rap pop dance fusion
                              ...                        
1059                                   pop uk dance funky
1060         worship rock music christian alternative ccm
1061                                           neo mellow
1062                             australian folkpop indie
1063                             punk rock emo pop modern
Name: artist_genre, Length: 1064, dtype: object

In [394]:
for i in string_field:
    print(set(i.split()))

{'hop', 'rap', 'urban', 'soul', 'contemporary', 'rhythm', 'hip', 'pop', 'neo', 'dance', 'blues', 'virginia'}
{'pop', 'dance'}
{'pop', 'rhythm', 'blues'}
{'pop', 'dance'}
{'reggae', 'rap', 'pop', 'dance', 'fusion'}
{'hop', 'carolina', 'rap', 'urban', 'south', 'contemporary', 'rhythm', 'hip', 'pop', 'atl', 'dance', 'blues'}
{'hop', 'carolina', 'rap', 'urban', 'south', 'contemporary', 'rhythm', 'hip', 'pop', 'atl', 'dance', 'blues'}
{'hop', 'carolina', 'rap', 'urban', 'south', 'contemporary', 'rhythm', 'hip', 'pop', 'atl', 'dance', 'blues'}
{'pop', 'group', 'dance', 'girl'}
{'urban', 'contemporary', 'rhythm', 'group', 'girl', 'pop', 'dance', 'blues'}
{'atlanta', 'hop', 'rap', 'south', 'old', 'dirty', 'hip', 'southern', 'atl', 'school'}
{'pop', 'latin', 'canadian', 'dance'}
{'pop', 'dance', 'postteen'}
{'pop', 'dance', 'postteen'}
{'pop', 'dance', 'postteen'}
{'rap', 'urban', 'contemporary', 'rhythm', 'hip', 'pop', 'dance', 'blues'}
{'hop', 'rap', 'urban', 'contemporary', 'rhythm', 'hip', 

In [395]:
#check_df['All_text'] = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
check_df['All_text'] = check_df.artist_genre
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.0471,0.810,125.461,226864,4,69,hop rap urban contemporary virginia hip pop ne...,72,62,hop rap urban contemporary virginia hip pop ne...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.2420,0.924,143.040,198800,4,84,pop dance,80,78,pop dance
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.0521,0.817,100.972,267267,4,79,pop dance,79,77,pop dance
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.3130,0.654,94.759,227600,4,4,reggae rap pop dance fusion,73,1,reggae rap pop dance fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Clean Bandit,spotify:track:5HuqzFfq2ulY1iBAW5CxLe,Rather Be (feat. Jess Glynne),I Cry When I Laugh,Mom's playlist,17,NaN,0.799,0.586,11,...,0.1930,0.549,120.970,227833,4,51,pop uk dance funky,74,67,pop uk dance funky
1060,Jason Gray,spotify:track:5eqz8gTDU13s7x2DZKiQA9,Remind Me Who I Am,A Way To See In The Dark,Mom's playlist,17,NaN,0.543,0.668,0,...,0.3220,0.428,93.009,227267,4,49,worship rock music christian alternative ccm,49,49,worship rock music christian alternative ccm
1061,Mary Lambert,spotify:track:3V3iy4K6paycRmTyrjQVwi,Secrets,Heart On My Sleeve,Mom's playlist,17,NaN,0.789,0.555,0,...,0.2150,0.713,93.229,223405,4,46,neo mellow,49,44,neo mellow
1062,Sheppard,spotify:track:0qt5f5EL92o8Snzopsv0en,Geronimo,Bombs Away,Mom's playlist,17,NaN,0.705,0.780,7,...,0.1150,0.457,142.028,218228,4,72,australian folkpop indie,61,65,australian folkpop indie


In [396]:
# Imports
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [397]:
# import nltk
# from nltk.stem import WordNetLemmatizer

In [398]:
# stopwords = nltk.corpus.stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# nltk.download('stopwords')

In [399]:
check_df.keys()

Index(['track_artist_name', 'track_track_uri', 'track_track_name',
       'track_album_name', 'playlist_name', 'playlist_pid',
       'playlist_description', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity', 'All_text'],
      dtype='object')

In [400]:
song_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity']
df_song_features = check_df[song_features]

In [401]:
df_song_features

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_popularity,artist_popularity,album_popularity
0,0.904,0.813,4,-7.105,0.1210,0.031100,0.006970,0.0471,0.810,125.461,69,72,62
1,0.774,0.838,5,-3.914,0.1140,0.024900,0.025000,0.2420,0.924,143.040,84,80,78
2,0.664,0.758,2,-6.583,0.2100,0.002380,0.000000,0.0598,0.701,99.259,21,87,17
3,0.892,0.714,4,-6.055,0.1410,0.201000,0.000234,0.0521,0.817,100.972,79,79,77
4,0.853,0.606,0,-4.596,0.0713,0.056100,0.000000,0.3130,0.654,94.759,4,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,0.799,0.586,11,-6.735,0.0377,0.162000,0.000002,0.1930,0.549,120.970,51,74,67
1060,0.543,0.668,0,-7.396,0.0272,0.000724,0.000080,0.3220,0.428,93.009,49,49,49
1061,0.789,0.555,0,-5.900,0.0410,0.026000,0.000135,0.2150,0.713,93.229,46,49,44
1062,0.705,0.780,7,-6.267,0.0805,0.456000,0.001520,0.1150,0.457,142.028,72,61,65


In [402]:
song_features_array = np.array(df_song_features)

In [403]:
from sklearn.decomposition import TruncatedSVD

In [404]:
vectorizer = TfidfVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(string_field)
print(X_train[0])
x_train_array = X_train.toarray()

svd = TruncatedSVD(n_components=1)
#print(type(svd))
text_X_train = svd.fit_transform(X_train)
#print(text_X_train)
#print(text_X_train.shape)

for index, value in enumerate(text_X_train):
    print(string_field[index], value)

print(text_X_train)
#print(x_train_array)
print(x_train_array.shape)
print(x_train_array[0][44])
f_xtrain_nosvp = np.concatenate((song_features_array, x_train_array), axis=1)
print('xtrain_nosvp', f_xtrain_nosvp.shape)
f_xtrain = np.concatenate((song_features_array, text_X_train), axis=1)
print(f_xtrain)
print(f_xtrain.shape)

#print(X_train)
#print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")

  (0, 340)	0.3089593252391149
  (0, 39)	0.2562885701625822
  (0, 306)	0.26360171603979426
  (0, 81)	0.22335989626899352
  (0, 237)	0.2914294079198019
  (0, 275)	0.12628232335540676
  (0, 165)	0.22674280761643964
  (0, 377)	0.5251180619872103
  (0, 75)	0.29431601520097356
  (0, 373)	0.33216337139635027
  (0, 298)	0.2201285103883751
  (0, 168)	0.21455583930739394
hop rap urban contemporary virginia hip pop neo dance rhythm blues soul [0.28958528]
pop dance [0.46481724]
pop rhythm blues [0.26755566]
pop dance [0.46481724]
reggae rap pop dance fusion [0.25121289]
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319]
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319]
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319]
pop group dance girl [0.22759885]
urban contemporary group girl pop dance rhythm blues [0.23292275]
atlanta hop rap south old dirty hip southern atl school [0.07713528]
pop lat

In [92]:
print(f"{X_train.nnz / np.prod(X_train.shape):.3f}")
print(f"About {X_train.nnz / np.prod(X_train.shape)*100:.3f}\% of the entries in the matrix are non zero")

0.077
About 7.683\% of the entries in the matrix are non zero


In [405]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(f_xtrain_nosvp)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], value)
    if index == 10:
        break
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

hop rap urban contemporary virginia hip pop neo dance rhythm blues soul 2
pop dance 2
pop rhythm blues 7
pop dance 9
reggae rap pop dance fusion 7
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 9
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 9
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 9
pop group dance girl 3
urban contemporary group girl pop dance rhythm blues 2
atlanta hop rap south old dirty hip southern atl school 9
Top terms per cluster
Cluster 0: alternative america americana ambient aesthetic adult acoustic alt alabama albuquerque 
Cluster 1: alternative america americana ambient aesthetic adult acoustic alt album albuquerque 
Cluster 2: alternative america americana ambient aesthetic adult acoustic alt albuquerque protopunk 
Cluster 3: alternative america americana ambient aesthetic adult alt acoustic albuquerque alabama 
Cluster 4: alternative america americana ambient aesthetic adult acoustic 

In [406]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(X_train)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], value)
    if index == 10:
        break
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

hop rap urban contemporary virginia hip pop neo dance rhythm blues soul 1
pop dance 4
pop rhythm blues 1
pop dance 4
reggae rap pop dance fusion 4
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 1
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 1
hop carolina rap urban south contemporary hip pop atl dance rhythm blues 1
pop group dance girl 4
urban contemporary group girl pop dance rhythm blues 1
atlanta hop rap south old dirty hip southern atl school 0
Top terms per cluster
Cluster 0: hip rap hop trap pop southern melodic conscious coast rouge 
Cluster 1: rhythm blues contemporary urban rap soul pop dance hip hop 
Cluster 2: postgrunge rock pop alternative metal grunge nu mellow neo funk 
Cluster 3: trip downtempo electronica hop instrumental hip jazztronica indietronica jazz ambient 
Cluster 4: dance pop rap miami postteen australian art hip hop indie 
Cluster 5: kpop group boy korean krap rhythm blues girl pop singersongwriter 
Clust

In [407]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(f_xtrain)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], text_X_train[index], value)
    if index == 10:
        break
    
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

hop rap urban contemporary virginia hip pop neo dance rhythm blues soul [0.28958528] 1
pop dance [0.46481724] 1
pop rhythm blues [0.26755566] 7
pop dance [0.46481724] 8
reggae rap pop dance fusion [0.25121289] 7
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319] 8
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319] 8
hop carolina rap urban south contemporary hip pop atl dance rhythm blues [0.25832319] 8
pop group dance girl [0.22759885] 4
urban contemporary group girl pop dance rhythm blues [0.23292275] 1
atlanta hop rap south old dirty hip southern atl school [0.07713528] 8
Top terms per cluster
Cluster 0: alternative america americana ambient aesthetic adult acoustic alt album albuquerque 
Cluster 1: alternative america americana ambient aesthetic adult acoustic alt anarchopunk albuquerque 
Cluster 2: alternative america americana ambient aesthetic adult acoustic alt alabama album 
Cluster 3: alternative america ame

In [364]:
#print(list(test_song[:1]))
#print(list(test_song)[0])
X_test = vectorizer.transform(list(test_song))
#print(X_test)
print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")
# feature_names = vectorizer.get_feature_names_out()
# print(feature_names)
prediction = model.predict(X_test)
print(prediction)

n_samples: 40, n_features: 92


ValueError: X has 92 features, but KMeans is expecting 14 features as input.

In [107]:
print(model.labels_)
print(prediction)

[4 2 2 4 2 2 0 2 2 2 2 4 4 4 1 3 3 3 3 3 3 3 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 2 4 4 0 2 2 0 2 2 2 4 4 2 0 2 2
 2 0 1]


In [108]:
# # Save machine learning model
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [109]:
# # load model only once
# with open('finalized_model.sav', 'rb') as fid:
#     model = pickle.load(fid)

In [110]:
check_df['ClusterPrediction'] = ""
check_df['All_text']

0      dance pop hip hop hip pop neo soul pop rap r&b...
1                                          dance pop pop
2                                                pop r&b
3                                          dance pop pop
4                        dance pop pop rap reggae fusion
                             ...                        
97                             k pop k pop boy group pop
98                                k pop k pop girl group
99                                                 k pop
100                                                k pop
101                                                k pop
Name: All_text, Length: 102, dtype: object

In [111]:
def cluster_predict(str_input):
    #print(test_song)
    if isinstance(str_input, str):
        # It is a single string element thus we cannot use list() and have to use brackets [] to transform to list
        X_test = vectorizer.transform([str_input])
        prediction = model.predict(X_test)
    else:
        # if is not a single string element so we can transform into a list directly using list()
        X_test = vectorizer.transform(list(str_input))
        prediction = model.predict(X_test)
    return int(prediction)

In [112]:
check_df['All_text'][0]

'dance pop hip hop hip pop neo soul pop rap r&b rap urban contemporary virginia hip hop'

In [113]:
cluster_predict(check_df['All_text'][0])

2

In [114]:
cluster_predict([check_df['All_text'][0]])

2

In [115]:
result = check_df['All_text'].apply(lambda x: cluster_predict(x))
print(result)

0      2
1      2
2      2
3      2
4      2
      ..
97     2
98     2
99     2
100    2
101    2
Name: All_text, Length: 102, dtype: int64


In [116]:
# for i in check_df['All_text']:
#     pred = cluster_predict(i)
#     print(i, pred)
check_df['ClusterPrediction'] = check_df['All_text'].apply(lambda x: cluster_predict(x))

In [117]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text,ClusterPrediction
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62,dance pop hip hop hip pop neo soul pop rap r&b...,2
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.924,143.040,198800,4,84,dance pop pop,80,78,dance pop pop,2
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.701,99.259,235933,4,21,pop r&b,87,17,pop r&b,2
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.817,100.972,267267,4,79,dance pop pop,79,77,dance pop pop,2
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1,dance pop pop rap reggae fusion,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.460,106.992,274097,4,1,k pop k pop boy group pop,90,1,k pop k pop boy group pop,2
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.517,126.977,218475,4,41,k pop k pop girl group,41,41,k pop k pop girl group,2
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.371,123.773,288993,4,66,k pop,66,66,k pop,2
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.389,80.038,217861,4,66,k pop,66,66,k pop,2


In [663]:
# for seed in range(5):
#     model = KMeans(
#         n_clusters=true_k,
#         max_iter=500,
#         n_init=1,
#         random_state=seed,
#     ).fit(X)
#     cluster_ids, cluster_sizes = np.unique(model.labels_, return_counts=True)
#     print(f"Number of elements asigned to each cluster: {cluster_sizes}")
# print()

In [118]:
def recommend_util(artist_name, song_name):
    
    # Predict category of input string category
    chosen_song_df = check_df.loc[(check_df['track_artist_name'] == artist_name) & (check_df['track_track_name'] == song_name)]
    str_input = chosen_song_df.track_track_name.str.cat(" " + chosen_song_df.artist_genre)
        
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    
    temp_df = check_df.loc[check_df['ClusterPrediction'] == prediction_inp]
    new_temp_df = temp_df.sample(5)
    
    return chosen_song_df[['track_artist_name', 'track_track_name']], new_temp_df[['track_artist_name', 'track_track_name']]

In [119]:
song_choice = 2
print(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
temp_df = check_df.loc[(check_df['track_artist_name'] == check_df['track_artist_name'][0]) & (check_df['track_track_name'] == check_df['track_track_name'][0])]
string_input = temp_df.track_track_name.str.cat(" " + temp_df.artist_genre)
string_input
prediction_inp = cluster_predict(string_input)
print(prediction_inp)

Beyoncé Crazy In Love
2


In [120]:
song_choice = 1
original_song , recommended_songs = recommend_util(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
print(original_song)
print('\n')
print(recommended_songs)

  track_artist_name track_track_name
1    Britney Spears            Toxic


    track_artist_name  track_track_name
101             Ailee   I Will Show You
33               Iyaz            Replay
98            Lovelyz           Ah-Choo
14             Cassie            Me & U
19    Destiny's Child  Jumpin', Jumpin'


In [228]:
# Example of how to pull metadata from a single track 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

Row = 5 

track_uri = subset_df.iloc[Row]['track_track_uri']
artist_uri = subset_df.iloc[Row]['track_artist_uri']
album_uri = subset_df.iloc[Row]['track_album_uri']

print('ARTIST: ', subset_df.iloc[Row]['track_artist_name'])
print('ALBUM: ', subset_df.iloc[Row]['track_album_name'])
print('SONG: ', subset_df.iloc[Row]['track_track_name'])
track_popularity = spotify.track(track_uri)['popularity']
print('track_popularity: ', track_popularity)
track_results = spotify.audio_features(tracks=track_uri)
print('audio_features: ', track_results[0])
artist_results = spotify.artist(artist_uri)
print('artist_genre: ', artist_results['genres'])
print('artist_popularity: ', artist_results['popularity'])
artist_albums_results = spotify.artist_albums(artist_uri)
#print(artist_albums_results)
album_results = spotify.album(album_uri)
print('album_popularity: ',album_results['popularity'])
print('album_keys: ', album_results.keys())
# print(artist_results['genres'])
# print(artist_results['popularity'])

ARTIST:  Ron Pope
ALBUM:  The Bedroom Demos
SONG:  A Drop In The Ocean
track_popularity:  67
audio_features:  {'danceability': 0.447, 'energy': 0.393, 'key': 5, 'loudness': -8.65, 'mode': 1, 'speechiness': 0.038, 'acousticness': 0.785, 'instrumentalness': 0, 'liveness': 0.28, 'valence': 0.564, 'tempo': 73.139, 'type': 'audio_features', 'id': '5JDcQAztvZTIkrWoZihgvC', 'uri': 'spotify:track:5JDcQAztvZTIkrWoZihgvC', 'track_href': 'https://api.spotify.com/v1/tracks/5JDcQAztvZTIkrWoZihgvC', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5JDcQAztvZTIkrWoZihgvC', 'duration_ms': 220239, 'time_signature': 3}
artist_genre:  ['neo mellow', 'piano rock', 'viral pop']
artist_popularity:  53
album_popularity:  59
album_keys:  dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])


In [229]:
# Work only with important features
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_duration_ms', 'playlist_description']

In [ ]:
artist_count_df = subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort=False).value_counts(sort=False).reset_index(name="artist_count")

In [ ]:
artist_count_df

In [ ]:
subset_df[['playlist_name', 'track_artist_name']]

In [ ]:
subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort = False).value_counts(sort=False).reset_index(name="artist_count").info()

In [ ]:
music_df = pd.merge(subset_df, artist_count_df,  how='left', on = ['playlist_name','track_artist_name'])

In [ ]:
music_df

In [ ]:
# subset_df.join(.sum(), on='playlist_name', rsuffix='_count')
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1)

In [ ]:
important_features.append("artist_count")

In [ ]:
music_df[important_features]

In [ ]:
music_df['artist_fraction'] =  music_df['artist_count'] / music_df['playlist_num_tracks'] 
music_df

In [ ]:
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).value_counts() 

In [ ]:
vc = music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x), axis=1).value_counts()
vc[vc > 1].sum() - len(vc[vc > 1])

In [ ]:
# for i in df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique():
# #     print(i.split(',.'))
#     if len(i.split(',.')) == 2:
#         print(i)

In [ ]:
np.unique(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_track_name'])

In [ ]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [ ]:
music_df[music_df['playlist_num_tracks'] == 20][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][important_features].groupby('playlist_name')['track_artist_name'].value_counts()